# Test epitope correlations and harmonizing

First we create three models: two are identical, one has epitopes flipped relative to the others but listed in the same order, and the other has epitopes flipped and listed in the opposite order:

In [1]:
import copy

import pandas as pd

from polyclonal import Polyclonal


activity_wt_df = pd.DataFrame({"epitope": [1, 2], "activity": [2.0, 1.0]})

mut_escape_df = pd.DataFrame(
    {
        "mutation": [
            "M1C",
            "M1C",
            "G2A",
            "G2A",
            "A4K",
            "A4K",
            "A4L",
            "A4L",
            "A4Q",
            "A4Q",
        ],
        "epitope": [1, 2, 1, 2, 1, 2, 1, 2, 1, 2],
        "escape": [2.0, 0.0, 3.0, 0.0, 0.0, 2.5, 0.0, 1.5, 0.0, 3.5],
    }
)

model = Polyclonal(mut_escape_df=mut_escape_df, activity_wt_df=activity_wt_df)

model_copy = copy.deepcopy(model)

model_flipped = Polyclonal(
    mut_escape_df=(
        mut_escape_df.assign(
            epitope=lambda x: x["epitope"].map({1: 2, 2: 1})
        ).sort_values("epitope")
    ),
    activity_wt_df=(
        activity_wt_df.assign(
            epitope=lambda x: x["epitope"].map({1: 2, 2: 1})
        ).sort_values("epitope")
    ),
)

model_flipped_diff_order = Polyclonal(
    mut_escape_df=(
        mut_escape_df.assign(epitope=lambda x: x["epitope"].map({1: 2, 2: 1}))
    ),
    activity_wt_df=(
        activity_wt_df.assign(epitope=lambda x: x["epitope"].map({1: 2, 2: 1}))
    ),
)

Now look at correlations among epitopes:

In [2]:
model.mut_escape_corr(model_copy)

,ref_epitope,self_epitope,correlation
0,1,1,1.000000
1,1,2,-0.860309
2,2,1,-0.860309
3,2,2,1.000000


In [3]:
model.mut_escape_corr(model_flipped)

,ref_epitope,self_epitope,correlation
0,1,1,-0.860309
1,1,2,1.000000
2,2,1,1.000000
3,2,2,-0.860309


In [4]:
model.mut_escape_corr(model_flipped_diff_order)

,ref_epitope,self_epitope,correlation
0,1,1,-0.860309
1,1,2,1.000000
2,2,1,1.000000
3,2,2,-0.860309


Now harmonize epitopes.
First, if we harmonize the identical models, nothing should change:

In [5]:
assert model.epitopes == model_copy.epitopes
assert model.epitope_colors == model_copy.epitope_colors
assert model.activity_wt_df.equals(model_copy.activity_wt_df)
assert model.mut_escape_df.equals(model_copy.mut_escape_df)

model_copy_harmonized, _ = model_copy.epitope_harmonized_model(model)

assert model.epitopes == model_copy_harmonized.epitopes
assert model.epitope_colors == model_copy_harmonized.epitope_colors
assert model.activity_wt_df.equals(model_copy_harmonized.activity_wt_df)
assert model.mut_escape_df.equals(model_copy_harmonized.mut_escape_df)

Now harmonize model with different epitopes in same order:

In [6]:
assert model.epitopes == model_flipped.epitopes
assert model.epitope_colors == model_flipped.epitope_colors
assert not model.activity_wt_df.equals(model_flipped.activity_wt_df)
assert not model.mut_escape_df.equals(model_flipped.mut_escape_df)

model_flipped_harmonized, _ = model_flipped.epitope_harmonized_model(model)

assert model.epitopes == model_flipped_harmonized.epitopes
assert model.epitope_colors == model_flipped_harmonized.epitope_colors
assert model.activity_wt_df.equals(model_flipped_harmonized.activity_wt_df)
assert model.mut_escape_df.equals(model_flipped_harmonized.mut_escape_df)

Now harmonize model with different epitopes in same order:

In [7]:
assert model.epitopes != model_flipped_diff_order.epitopes
assert model.epitope_colors != model_flipped_diff_order.epitope_colors
assert not model.activity_wt_df.equals(model_flipped_diff_order.activity_wt_df)
assert not model.mut_escape_df.equals(model_flipped_diff_order.mut_escape_df)

(
    model_flipped_diff_order_harmonized,
    _,
) = model_flipped_diff_order.epitope_harmonized_model(model)

assert model.epitopes == model_flipped_diff_order_harmonized.epitopes
assert model.epitope_colors == model_flipped_diff_order_harmonized.epitope_colors
assert model.activity_wt_df.equals(model_flipped_diff_order_harmonized.activity_wt_df)
assert model.mut_escape_df.equals(model_flipped_diff_order_harmonized.mut_escape_df)